In [ ]:
library(DESeq2)
library(tidyverse)
library(genefilter)
library(pheatmap)
library(sva)
library(RColorBrewer)
library(apeglm)

## Part 5: Making the Data Meaningful

Welcome to **R**! In the last notebook, data was retrieved from the sample, converted into quantitative measurements, then assembled into a summary counts table. Let's take a look at the current shape of the data:

In [ ]:
data <- read.csv("./data/counts_table.tsv", header=TRUE, sep="\t",  row.names = 1)
data

While many genes have reads, some have no reads, making this a matrix full of many zero values, or what is known as a **sparse matrix**. These zeros can make it difficult to analyze the data, as sometimes a zero value doesn't truly mean there was no expression in a sample for a gene -- just that it didn't get picked up by the sequencer. For that reason, we have to remove all of the genes that have no values at all.

In [ ]:
data <- as.matrix(data)
condition <- factor(c(rep("Basal", 3), rep("Luminal", 3)))
coldata <- data.frame("replicate" = colnames(data), condition)
deseqrna <- DESeqDataSetFromMatrix(countData = data, colData = coldata, design = ~condition)
deseqrna <- deseqrna[ rowSums(counts(deseqrna)) > 1, ]
counts(deseqrna)

Once trimmed down, the data has gone from 27179 genes to 13860. This is a good start, but now the data needs to be standardized againsnt outlier values

In [ ]:
variance <- vst(deseqrna, blind = T)

Finally, it's time to view our data. Don't worry about the math that goes into these for now, just recongnize that, without the programs that have been created for these ananlyses, getting these results would be next to impossible.

## Comparison between samples

This graph shows how similar each of the samples are to one another. The darkest red diagonal shows that the sets are identical -- which makes sense since it's being compared to itself.

In [ ]:
rnadist <- dist(t(assay(variance)))
rnadistmatrix <- as.matrix(rnadist)
rownames(rnadistmatrix) <- paste(variance$dex, variance$cell, sep = " - ")
colnames(rnadistmatrix) <- NULL
colors <- colorRampPalette( rev(brewer.pal(9, "Reds")) )(255)
pheatmap(rnadistmatrix,
         clustering_distance_rows = rnadist,
         clustering_distance_cols = rnadist,
         col = colors,
         cellheight = 35, cellwidth = 35,
         main = "Sample-to-sample distances")

Multidimensional scalign is complicated, but boils down to taking all of the differences between data points and represeting as best as possible in two or three dimensions. This graph shows that the Basal samples are more similar to one another than the luminal samples, since their  points are more closely clustered.

In [ ]:
pca <- plotPCA(variance, intgroup = c("replicate", "condition"), returnData = T)
pvar <- round(100 * attr(pca, "percentVar"))
ggplot(pca, aes(x = PC1, y = PC2, color = condition)) + geom_point() +
  xlab(paste0("PC1: ", pvar[1], "% variance")) +
  ylab(paste0("PC2: ", pvar[2], "% variance")) +
  coord_fixed()

This next graph shows the actual change in gene expression between the Basal and Luminal datasets, with genes that have real differences in expression levels, not just the natural fluctuations, highlighted in red. The gene circled and labeled is the most differentially expressed gene between the two.

In [ ]:
deseqsetrna <- DESeq(deseqrna) 
res <- results(deseqsetrna, alpha = 0.05,  contrast = c("condition", "Basal", "Luminal"))
ressub <- subset(res, padj <= 0.05)
mostsig <- rownames(res)[which.min(res$padj)]
reslfc <- lfcShrink(deseqsetrna, coef="condition_Luminal_vs_Basal", type="apeglm")
plotMA(reslfc, ylim = c(-10,10), colSig="red")
title("Altered Regulation in Basal vs Luminal Cells")
with(res[mostsig, ], {
  points(baseMean, log2FoldChange, col="dodgerblue", cex=2, lwd=2)
  text(baseMean, log2FoldChange, mostsig, pos=2, col="dodgerblue")
})

Finally, let's look at how many genes are differentially expressed. A p-value denotes **significance**. p represents the likelyhood the null hypothesis is true, that two datasets are actually subsets of the same set, in percent. When p <0.05, it is understood that there is a real difference between the compared data, and less than a 5% chance that the difference was just due to lucky sampling. The method of calculating p is different depending on the type of data.

In [ ]:
hist(res$pvalue[res$baseMean > 1], breaks = 0:20/20,
     col = "purple", border = "white", main = "Distrubution of p-Values",
     xlab = "pval")

### Question: About how many genes are significantly differentially expressed?

# Congratulations, you've completed the demo!
## Now let's hear from you! Any questions?